In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
columns = ['id', 'year', 'category', 'nominee', 'staff', 'company', 'producer', 'win']
url = 'https://www.emmys.com/awards/nominations/award-search?page={page}'

In [ ]:
has_page = True
page = 0
id_ = 1
last_year = 2000
row_list = []
error_list = []

while has_page:
    req = requests.get(url.format(page=page))
    
    if req.status_code == 200:
        soup = BeautifulSoup(req.content, 'html.parser')
        results = soup.find(class_='awards-search-page')
        rows = results.find_all(class_='views-row')
        for row in rows:
            info = row.find_all('li')
            
            if len(info) < 3:
                error_list.append((page, id_))
                continue
                
            wrap = row.find(class_='wrap')
            
            emmy_category = wrap.h3.text  
                
            if row.find('ul', class_='winner'):
                emmy_winner = True
            else:
                emmy_winner = False
                
            emmy_nominee = info[-3].text
            emmy_company = info[-2].text
            emmy_producer = info[-1].text
            
            if info[-4]:
                emmy_staff = '; '.join([staff.text for staff in info[:-3] if staff.text != 'Nominee' if staff.text != 'Winner'])
            
            emmy_year = emmy_category[-4:]
            emmy_category = emmy_category.replace(f' - {emmy_year}', '')
            
            row_list.append([id_, emmy_year, emmy_category, emmy_nominee, emmy_staff, emmy_company, emmy_producer, emmy_winner])
            
            id_ += 1
            
            if emmy_year != last_year:
                last_year = emmy_year
                print(page, emmy_year)
    
        page += 1
            
    else:
        has_page = False
        print(req.status_code)

0 2019
67 2018
134 2017
200 2016
261 2015
320 2014
378 2013
434 2012
488 2011
540 2010
594 2009
646 2008
695 2007
742 2006
789 2005
834 2004


In [ ]:
df = pd.DataFrame(dict(zip(columns, row_list)), columns=columns)

In [ ]:
len(row_list)

In [ ]:
row_list[-1]

In [ ]:
error_list